# Where is Rideshare the Most Lucrative?
<p>A look at the [U.S. Census Bureau's Nonemployer Statistics]("https://www.census.gov/programs-surveys/nonemployer-statistics/data/datasets.html?eml=gd&utm_medium=email&utm_source=govdelivery") for metropolitan areas in 2016.

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

### Read in the Data
<p>Now, let's read in the county nonemployer business data for 2016.</p>
<p>We will also need to read in a separate database with metro names to join to our full database.</p>

In [2]:
bus2016 = pd.read_csv("/Users/alexmahadevan/Desktop/uber-change/nonemp16msa.txt")
metro_names = pd.read_csv("/Users/alexmahadevan/Desktop/uber-change/metro-fips.csv", encoding="ISO-8859-1")

### Inspect the Data
<p>Let's have a look at both datasets.</p>

In [3]:
bus2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349296 entries, 0 to 349295
Data columns (total 7 columns):
MSA           349296 non-null int64
NAICS         349296 non-null object
ESTAB_F       83279 non-null object
ESTAB         349296 non-null int64
RCPTOT_N_F    349296 non-null object
RCPTOT_F      83279 non-null object
RCPTOT        349296 non-null int64
dtypes: int64(3), object(4)
memory usage: 18.7+ MB


In [4]:
metro_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1899 entries, 0 to 1898
Data columns (total 2 columns):
MSA     1899 non-null int64
NAME    1899 non-null object
dtypes: int64(1), object(1)
memory usage: 29.8+ KB


### Clean and Filter the Data
<p>We will have to removed some duplicates from the metro names list and filter for only "taxi and limousine services".</p> 

In [5]:
metro_names_deduplicated = metro_names.drop_duplicates()

In [6]:
metro_names_deduplicated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 945 entries, 0 to 1898
Data columns (total 2 columns):
MSA     945 non-null int64
NAME    945 non-null object
dtypes: int64(1), object(1)
memory usage: 22.1+ KB


In [7]:
uber2016 = bus2016[bus2016["NAICS"] == "4853"]

In [8]:
len(uber2016)

904

<p>We should remove any bad data. Luckily, the Census Bureau flags observations that might have a high level of noise.</p>

In [9]:
uber2016 = uber2016[uber2016["RCPTOT_N_F"] != "J"]

In [10]:
len(uber2016)

762

<p>Now let's join our datasets.</p>

In [11]:
joined = pd.merge(uber2016, metro_names_deduplicated, on="MSA")

In [12]:
len(joined)

762

In [13]:
joined.head()

,MSA,NAICS,ESTAB_F,ESTAB,RCPTOT_N_F,RCPTOT_F,RCPTOT,NAME
0,10100,4853,S,0,S,S,0,"Aberdeen, SD"
1,10180,4853,NaN,52,H,NaN,415,"Abilene, TX"
2,10220,4853,D,0,D,D,0,"Ada, OK"
3,10300,4853,NaN,25,G,NaN,833,"Adrian, MI"
4,10420,4853,NaN,557,G,NaN,4677,"Akron, OH"


<p>Now, we're going to simplify the dataset. We only need a few of these fields.</p>

In [20]:
uber_joined = joined[["MSA", "NAME", "RCPTOT", "ESTAB"]]

In [21]:
uber_joined.head()

,MSA,NAME,RCPTOT,ESTAB
0,10100,"Aberdeen, SD",0,0
1,10180,"Abilene, TX",415,52
2,10220,"Ada, OK",0,0
3,10300,"Adrian, MI",833,25
4,10420,"Akron, OH",4677,557


### Analyze the Data
<p>Now we get to the fun part. Let's create a new variable that shows the average earnings among drivers ("ESTAB"). We divide the total receipts collected in the metro area by the number of drivers.</p>

In [24]:
uber_joined = uber_joined.copy()

In [25]:
uber_joined["per_capita"] = uber_joined["RCPTOT"] / uber_joined["ESTAB"]

In [46]:
uber2016_sorted = uber_joined.sort_values("per_capita", ascending=False)

<p>Now we send the unadjusted earnings to our folder.</p>

In [44]:
uber2016_sorted.to_csv("/Users/alexmahadevan/Desktop/uber-change/uber_2016_sorted_earnings_msa.csv")

<p>But wait, we need to adjust for the cost of living of these areas! Let's use regional price parities from the [U.S. Bureau of Economic Analysis]("https://www.bea.gov") to do this. </p>

In [54]:
rpp = pd.read_csv("/Users/alexmahadevan/Desktop/uber-change/rpp_msa.csv")

In [55]:
rpp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 2 columns):
MSA    383 non-null int64
rpp    383 non-null float64
dtypes: float64(1), int64(1)
memory usage: 6.1 KB


<p>Now we merge the datasets.</p>

In [57]:
adjusted = pd.merge(uber2016_sorted, rpp, on="MSA")

In [59]:
adjusted.head()

,MSA,NAME,ESTAB,RCPTOT,per_capita,rpp
0,16940,"Cheyenne, WY",11,1070,97.272727,97.4
1,20020,"Dothan, AL",17,1003,59.000000,84.1
2,33740,"Monroe, LA",40,2180,54.500000,84.7
3,20220,"Dubuque, IA",38,1877,49.394737,91.5
4,33220,"Midland, MI",8,380,47.500000,91.8


<p>Now we add the deflator to control for the local prices.</p>

In [82]:
adjusted["per_capita_adj"] = adjusted["per_capita"] / (adjusted["rpp"] / 100)

In [83]:
adjusted.head()

,MSA,NAME,ESTAB,RCPTOT,per_capita,rpp,per_capita_adj
0,16940,"Cheyenne, WY",11,1070,97.272727,97.4,99.869330
1,20020,"Dothan, AL",17,1003,59.000000,84.1,70.154578
2,33740,"Monroe, LA",40,2180,54.500000,84.7,64.344746
3,20220,"Dubuque, IA",38,1877,49.394737,91.5,53.983319
4,33220,"Midland, MI",8,380,47.500000,91.8,51.742919


</p>Again, send it to our folder.</p>

In [64]:
adjusted.to_csv("/Users/alexmahadevan/Desktop/uber-change/uber_2016_sorted_earnings_msa_adjusted.csv")

<p>Finally, we need to find the median number of drivers to establish a cutoff for our list.</p>

In [84]:
adjusted["ESTAB"].describe()

count       363.000000
mean       1901.220386
std        8732.078626
min           8.000000
25%          45.500000
50%         146.000000
75%         608.000000
max      131778.000000
Name: ESTAB, dtype: float64

In [85]:
adjusted_trimmed = adjusted[adjusted["ESTAB"] > 146]

Now, we have our final list! Make sure to check out [The Penny Hoarder]("https://www.thepennyhoarder.com") for more ways to put more money in your pocket!